Csv "food.csv" "food_nutrient.csv" "nutrient.csv" "food_category" are from USDA SR Legacy Foods Dataset. 
All Csv created i moved to Csv_Data folder.

Step #1 Load food.csv and take all the words from all description (food names) and create new csv ordered by frequency of word spoken. Make all words lowercase for smoother proccesing.

In [59]:
import pandas as pd
import re
from collections import Counter
import sqlite3

In [60]:
raw_foods_df = pd.read_csv('Csv_Data/food.csv')
raw_foods_df = raw_foods_df.drop_duplicates(subset='description')
raw_foods_df = raw_foods_df.dropna(subset='description')
raw_foods_df['description'] = raw_foods_df['description'].str.lower()
# taking out babyfood category
raw_foods_df = raw_foods_df[~raw_foods_df['food_category_id'].isin([3])]

In [61]:
all_words = " ".join(raw_foods_df['description'].astype(str)).lower()
any_word_regex = r'\b[a-z,0-9]{2,}\b'
raw_foods_words = re.findall(any_word_regex,all_words)
count = Counter(raw_foods_words)
keywords = [[word, count] for word ,count in count.items()]
all_words_df = pd.DataFrame(keywords,columns=['words','count']).sort_values(by='count',ascending=False)
all_words_df.to_csv('all_words.csv')

Step #2 Open up all_words.csv and go over the first 500 most spoken words by hand and decide which words we want to add to our filter to filter out. I created a new csv and typed the words i wanted to filter inside junk_words.csv. After that i made a category column and put the words i wanted to filter into two different categories junk and filler. Junk is words i want to remove from the description columns completely think words like imported, its not really useful to get the macros for the food and is just extra messy string data for UI, the other group is filler which is for words like and I dont want to remove the word and completely but i also dont want to add it into our prefix tree later on so i put the filler words in a category as well.

In [62]:
# Csv with junk words in category filler or junk
junk_words_df = pd.read_csv('Csv_Data/junk_words.csv')
 

Step #3 Clean up food descriptions with the words from our filter and clean up any columns or hyphens that are left afterwards.

In [63]:
junk_words = junk_words_df[junk_words_df['category'] == 'junk']
filler_words = junk_words_df[junk_words_df['category'] == 'filler']
junk_words_regex = r'\b(?:' + "|".join(junk_words['words']) + r')\b'  

In [64]:
# Remove junk words
raw_foods_df['description'] = raw_foods_df['description'].str.replace(junk_words_regex,"",regex=True)

# Remove excess commas and hyphens
raw_foods_df['description'] = raw_foods_df['description'].str.replace(r'([,-]\s*){2,}', ', ', regex=True)

# Replace excess whitespace with one space
raw_foods_df['description'] = raw_foods_df['description'].str.replace(r'\s{2,}', ' ', regex=True)

# Removes any combination of commas, hyphens, or spaces  trailing at the end or front 
raw_foods_df['description'] = raw_foods_df['description'].str.lstrip(',- ')
raw_foods_df['description'] = raw_foods_df['description'].str.rstrip(',- ')

# Removes parenthesis and its contents inside if the first character inside parenthesis is a space
raw_foods_df['description'] = raw_foods_df['description'].str.replace(r'\s\(\s+.*?\)', '', regex=True)

Step #4 Add nutrient data to raw foods csv and create updated table replacing old one. Inside nutrient.csv it will show nutrient id for nutrient name and the id associated with the nutrient in food_nutrient.csv we will be connecting each food nutrient value with the fdc_id then putting the measurement inside its own column labeled as the appropriate nutrient. Each nutrient is per 100g of said food and each nutrient is in column amount inside food_nutrient.csv so we will add one by one each nutrient changing the name of column amount to the nutrients name.

In [65]:
protein_id = 1003
fat_id = 1004
carbs_id = 1005
calories_id = 1008

In [66]:
food_nutrient_df = pd.read_csv('Csv_Data/food_nutrient.csv')
# Make filter for each nutrient
protein_mask = food_nutrient_df[food_nutrient_df['nutrient_id'] == protein_id]
protein_df = protein_mask[['fdc_id','amount']]

fat_mask = food_nutrient_df[food_nutrient_df['nutrient_id'] == fat_id]
fat_df = fat_mask[['fdc_id','amount']]

carbs_mask = food_nutrient_df[food_nutrient_df['nutrient_id'] == carbs_id]
carbs_df = carbs_mask[['fdc_id','amount']]

calories_mask = food_nutrient_df[food_nutrient_df['nutrient_id'] == calories_id]
calories_df = calories_mask[['fdc_id','amount']]

Merge each nutrient as a data frame with a left join to not lose any data in raw foods table, save to new csv.

In [67]:
result_protein = raw_foods_df.merge(protein_df,on='fdc_id',how='left')
result_protein.rename(columns={'amount': 'protein'},inplace= True)

result_fat = result_protein.merge(fat_df,on='fdc_id',how='left')
result_fat.rename(columns={'amount': 'fat'},inplace= True)

result_carbs = result_fat.merge(carbs_df,on='fdc_id',how='left')
result_carbs.rename(columns={'amount': 'carbs'},inplace= True)

result_calories = result_carbs.merge(calories_df,on='fdc_id',how='left')
result_calories.rename(columns={'amount': 'calories'},inplace= True)

# Fill missing columns with 0 and missing calorie columns with Atwater calculation from the other nutrients
 
for col in ['protein', 'fat', 'carbs']:
    result_calories[col] = result_calories[col].fillna(0)

result_calories['calories'] = result_calories['calories'].fillna(
    result_calories['protein']*4 + result_calories['fat']*9 + result_calories['carbs']*4
)
# Keep only neccasary columns data type and publication date are not needed
final_results = result_calories[['fdc_id','description','food_category_id','protein','fat','carbs','calories']]
final_results.to_csv('food_macros.csv')

Step #5 Create food word dictionary of all revelvant search words so we can create a inverted index for our search engine and connect words to documents. For each word we will create a row with the word and fdc_id and categry weight to measure word importance. Create new csv at the end to use for our Search engine file.

In [68]:
# Remove junk and filler words from all_words.csv and create new csv keyword.csv that contains only relevant words

keywords_df = all_words_df[~all_words_df['words'].isin(junk_words_df['words'])]
keywords_df['words'].to_csv('keywords.csv')

In [69]:
# Open food_category.csv and look for what groups you want to prioritize and give a weight value to the group there is 28 categories
high_priority = [1,2,4,5,9,10,11,12,13,15,16,17]


In [70]:
# Load both csv's we created earlier the new table and the table with all the words in description
food_macros_df = pd.read_csv('Csv_Data/food_macros.csv')
keywords_df = pd.read_csv('Csv_Data/keywords.csv') 


food_dictionary_df = pd.DataFrame(columns={'words': None,'fdc_id': None,'weight': None})
# Add | the or operator to each word inside your join statement
regex_dictionary_words = r'\b(?:' + "|".join(keywords_df['words'].astype(str)).lower() + r')\b'

# For each row in the food macros we will parse each word that is apart of our recognized word list
# For each word in list we will add new entry to dictionary with word and the rows id
# Use itertuples to access each rows properties easily
for row in food_macros_df.itertuples():
    weight = .5
    if row.food_category_id in high_priority:
        weight = 1.5 
    words = row.description
    try:
        parsed_words = re.findall(regex_dictionary_words,words)
        if len(parsed_words) > 0:
            for word in parsed_words:
                food_dictionary_df.loc[len(food_dictionary_df)] = [word, row.fdc_id, weight]
        else:
            print(row.description)

    except Exception as e:
        # I had one row that didnt pass the word screening and is just a "" blank white space fdc id 170471 i just deleted manually
        print(row.fdc_id)
        print(e)
    

food_dictionary_df.to_csv('food_dictionary.csv')


Step #6 Convert csv files into sqlite database to access some powerful query features and better portability to other devices

In [71]:
keywords_df = pd.read_csv('Csv_Data/keywords.csv')
food_dict_df = pd.read_csv('Csv_Data/food_dictionary.csv')
food_macros_df = pd.read_csv('Csv_Data/food_macros.csv')
conn = sqlite3.connect('search_engine.db')
keywords_df.to_sql('keywords_table',conn,index=False,if_exists='replace')
food_dict_df.to_sql('food_dict_table',conn,index=False,if_exists='replace')
food_macros_df.to_sql('food_macros_table',conn,index=False,if_exists='replace')

7447

In [ ]:
# Index Food dictionary on words for faster lookup times
cursor = conn.cursor()
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_food_dict_words
ON food_dict_table(words)     
               """)
conn.commit()


In [ ]:
# Check if index is created should print out index list idx_food_dict_words will be there
cursor.execute("PRAGMA index_list(food_dict_table);")
print(cursor.fetchall())
# Check if sql is using index you should see using index idx_food_dict_words 
cursor.execute(f"""EXPLAIN QUERY PLAN
SELECT fdc_id
FROM food_dict_table
WHERE words = 'pillsbury';
""")
print(cursor.fetchall())

[(0, 'idx_food_dict_words', 0, 'c', 0)]
[(3, 0, 0, 'SEARCH TABLE food_dict_table USING INDEX idx_food_dict_words (words=?)')]


Step #7 Move over to the Search_Engine.py file to use this data to create our search engine!!